## MLP Smoothing / Merging Baseline

Train a baseline MLP model for smoothing using graph-level features.

In [3]:
import torch
from utils.data_utils import HGCALTracksterPairs
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader

In [18]:
transform = T.Compose([T.NormalizeFeatures()])

# the first run includes building the edges and takes long
# no negative samples for pion are available for now
ds = HGCALTracksterPairs("data", kind="photon", transform=transform)

In [22]:
# balance the dataset
pos = ds[ds.data.y == 1]
neg = ds[ds.data.y == 0]
len_neg = len(neg)
len_pos = len(pos)
print(f"Positive: {len_pos}, Negative {len_neg}")
shorter = min(len_neg, len_pos)
test_n = int(shorter / 10)

Positive: 7060, Negative 10784


In [24]:
train_set = pos[:shorter - test_n] + neg[:shorter - test_n]
test_set = pos[shorter - test_n:shorter] + neg[shorter - test_n:shorter]
print(f"Train samples: {len(train_set)}, Test samples: {len(test_set)}")
train_dl = DataLoader(train_set, batch_size=8, shuffle=True)
test_dl = DataLoader(test_set, batch_size=8, shuffle=True)

Train samples: 12708, Test samples: 1412


In [27]:
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


In [26]:
class BaselineMerger(torch.nn.Module):
    pass